In [9]:
#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#skip to the next cell to load the result from file

#E7 as a sublattice of E8 with x_1+..+x_8=0
#E7* is the union of E7 and the translate by [1]=(1/4,..,1/4,-3/4,-3/4)


n=7 #lattice dim
m=8 #space dim

R = sqrt(7/8) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
short = []
vshort = []
K = ceil(4*R+3/2) #[1] has norm 3/2
Kh = K 
if Kh%2==0:
    Kh+=1 #odd ceiling
K2 = (4*R)^2
Kv = (2*R)^2

#reduced form: x_1>=..>=x_6, x_7>=x_8

for x1 in range(1+K):
    for x2 in range(-K,1+x1):
        for x3 in range(-K,1+x2):
            for x4 in range(-K,1+x3):
                for x5 in range(-K,1+x4):
                    for x6 in range(-K,1+x5):
                        for x7 in range(-K,1+K):
                            x = [x1,x2,x3,x4,x5,x6,x7,-(x1+x2+x3+x4+x5+x6+x7)]
                            norm2 = sum(t^2 for t in x)
                            if norm2>0:
                                if norm2 <= Kv:
                                    vshort.append(x.copy())
                                if norm2 <= K2:
                                    short.append(x.copy())   
                            for i in range(6):
                                x[i] += 1/4
                            x[6] -= 3/4
                            x[7] -= 3/4
                            norm2 = sum(t^2 for t in x)
                            if norm2>0:
                                if norm2 <= Kv:
                                    vshort.append(x.copy())
                                if norm2 <= K2:
                                    short.append(x.copy())   
                            for i in range(6):
                                x[i] -= 1/2
                            x[6] += 3/2
                            x[7] += 3/2
                            norm2 = sum(t^2 for t in x)
                            if norm2>0:
                                if norm2 <= Kv:
                                    vshort.append(x.copy())
                                if norm2 <= K2:
                                    short.append(x.copy())   
                            
for x1 in range(-Kh,Kh+2,2):
    for x2 in range(-Kh,1+x1,2):
        for x3 in range(-Kh,1+x2,2):
            for x4 in range(-Kh,1+x3,2):
                for x5 in range(-Kh,1+x4,2):
                    for x6 in range(-Kh,1+x5,2):
                        for x7 in range(-Kh,2+Kh,2):
                            x = [x1/2,x2/2,x3/2,x4/2,x5/2,x6/2,x7/2,-(x1+x2+x3+x4+x5+x6+x7)/2]
                            norm2 = sum(t^2 for t in x)
                            if norm2>0:
                                if norm2 <= Kv:
                                    vshort.append(x.copy())
                                if norm2 <= K2:
                                    short.append(x.copy())           
                            for i in range(6):
                                x[i] += 1/4
                            x[6] -= 3/4
                            x[7] -= 3/4
                            norm2 = sum(t^2 for t in x)
                            if norm2>0:
                                if norm2 <= Kv:
                                    vshort.append(x.copy())
                                if norm2 <= K2:
                                    short.append(x.copy())   
                            for i in range(6):
                                x[i] -= 1/2
                            x[6] += 3/2
                            x[7] += 3/2
                            norm2 = sum(t^2 for t in x)
                            if norm2>0:
                                if norm2 <= Kv:
                                    vshort.append(x.copy())
                                if norm2 <= K2:
                                    short.append(x.copy())   
                                
                                
print("very short:",len(vshort))
print("short:",len(short))
                            

#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in [0,1,2,3,4,6]:
    toadd = [0]*9
    toadd[i+1]=1 #x_j >= x_{j+1} for j=1,..,7 except 6
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0,1,1,1,1,1,1,1,1],[0,-1,-1,-1,-1,-1,-1,-1,-1]] #we are in x_1+..+x_8=0

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    cnt = len(V.faces(d))
    for ff in V.faces(d):        
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

        

print("forbidden:",len(forb))

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t16 in Permutations(x[0:6]):
        for t78 in Permutations(x[6:8]):
            c = list(t16) + list(t78)
            forbd[str(c)]=0

print("forbidden without symmetries:",len(forbd))
###


very short: 71
short: 1655
forbidden: 581
forbidden without symmetries: 48568


In [1]:
#below can be used for saving/loading large data for example n=7
import pickle
# with open('E7star.p','wb') as f:
#     pickle.dump((short,forb,forbd),f)
    
with open('E7star.p','rb') as f:
    (short,forb,forbd) = pickle.load(f)
n=7




In [4]:
#converting to basis
M = matrix(QQ,[[-1,1,0,0,0,0,0],
           [0,-1,1,0,0,0,0],
           [0,0,-1,1,0,0,0],
           [0,0,0,-1,1,0,0],
           [0,0,0,0,-1,1,0],
           [0,0,0,0,0,-1,1],
           [-3/4,-3/4,1/4,1/4,1/4,1/4,1/4]])
Mi = M.inverse().transpose()
forbdv = []
for x in forbd:
    y = eval(x)
    if sum(y)>=0: #not all are needed due to central symmetry
        forbdv.append(Mi*vector(y[0:7]))

In [5]:
def check_sublattice(basis,target=Infinity): 
    A = matrix(basis).transpose() 

    Ad = abs(A.det())
    if Ad==0 or Ad>=target:
        return Infinity

    Ai = A.inverse()
    for x in forbdv:
        y = Ai*x
        allint = True
        for t in y:
            if t!=floor(t):
                allint = False
                break
        if allint:
            return Infinity   
    
    return Ad

In [6]:
check_sublattice([[0, -1, -2, -3, -1, -2, 0],
 [-4, -5, -2, -2, -1, 0, 4],
 [-5, -10, -9, -5, -4, -1, 6],
 [-3, -7, -4, -4, -1, 0, 4],
 [-4, -5, -5, -4, -3, 0, 4],
 [-4, -5, -4, -1, 0, 0, 4],
 [-1, -4, -4, -3, -3, 0, 4]])

1372